In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
!pip install tensorflow

In [ ]:

# === 1. Simulate Vasicek Data ===
def simulate_vasicek(a, b, sigma, r0, T, dt=1.0):
    steps = int(T / dt)
    rates = [r0]
    for _ in range(steps):
        dr = a * (b - rates[-1]) * dt + sigma * np.random.normal() * np.sqrt(dt)
        rates.append(rates[-1] + dr)
    return np.array(rates)

true_a, true_b, true_sigma = 0.1, 0.03, 0.01
r0 = 0.025
T = 499  # 500 data points
rates = simulate_vasicek(true_a, true_b, true_sigma, r0, T)

# === 2. Preprocessing ===
scaler = MinMaxScaler()
rates_scaled = scaler.fit_transform(rates.reshape(-1, 1))

sequence_length = 50
X, y = [], []

for i in range(len(rates_scaled) - sequence_length):
    X.append(rates_scaled[i:i+sequence_length])
    y.append([true_a, true_b, true_sigma])  # Known parameters for supervision

X = np.array(X)
y = np.array(y)

# Reshape for LSTM: (samples, timesteps, features)
X = X.reshape((X.shape[0], X.shape[1], 1))

# === 3. Define the Model ===
model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(sequence_length, 1)),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(3)  # Output: a, b, sigma
])

model.compile(optimizer='adam', loss='mse')
model.summary()

# === 4. Train the Model ===
history = model.fit(X, y, epochs=500, verbose=0)

# === 5. Predict Parameters ===
predicted_params = model.predict(X)
final_pred = predicted_params[-1]

print("\n--- Final Predicted Parameters ---")
print(f"a:      {final_pred[0]:.5f}")
print(f"b:      {final_pred[1]:.5f}")
print(f"sigma:  {final_pred[2]:.5f}")

# === 6. Plot loss curve ===
plt.plot(history.history['loss'])
plt.title("Training Loss")
plt.xlabel("Epoch")
plt.ylabel("MSE")
plt.show()